In [7]:
#PREPROCESS_make_3_omics_matrix.py
#merging three preprocessed data & patient information into a single data matrix.
#designed for network inference and machine learning

import pandas as pd

In [8]:
def file_to_dict(data_file, data_dict, common_feature_list, common_patientID_list, feature_header):
    
    data_df = pd.read_csv(data_file, sep="\t", index_col=0)
    feature_list = data_df.index.values
    patientID_list = data_df.columns.values
    r, c = data_df.shape
    
    #make patientID_list if not exist
    for patientID in patientID_list:
        if patientID not in common_patientID_list:
            common_patientID_list.append(patientID)
            
    for i in range(r):
        feature = feature_list[i]
        if feature_header != 'nan':
            feature = '%s_%s' % (feature_header, feature)
        if feature not in common_feature_list:
            common_feature_list.append(feature)
            
        for j in range(c):
            patientID = patientID_list[j]
            value = data_df.iloc[i][j]
            data_dict[feature, patientID] = value
            
    
    return common_feature_list, common_patientID_list, data_dict

In [9]:
output_dir ='../../../preprocessed_data/3_omics'

patient_info_file = '../../../preprocessed_data/meta/patient_info.ML_ready.tsv'
autoantibody_file = '../../../preprocessed_data/autoantibody/sengenics_qnorm_data.v2.tsv'
metabolomics_file = '../../../preprocessed_data/metabolomics/metabolone_raw_norm_preprocessed.v2.tsv'
proteomics_file = '../../../preprocessed_data/proteomics/somascan_anml.T.v2.tsv'



In [10]:
data_dict = {}
common_feature_list = []
common_patientID_list = []

common_feature_list, common_patientID_list, data_dict = file_to_dict(patient_info_file, data_dict, common_feature_list, common_patientID_list,'nan')
common_feature_list, common_patientID_list, data_dict = file_to_dict(autoantibody_file, data_dict, common_feature_list, common_patientID_list,'aa')
common_feature_list, common_patientID_list, data_dict = file_to_dict(metabolomics_file, data_dict, common_feature_list, common_patientID_list,'nan')
common_feature_list, common_patientID_list, data_dict = file_to_dict(proteomics_file, data_dict, common_feature_list, common_patientID_list,'p')


In [11]:
output_file = '%s/three_omics_multiplex.tsv' % output_dir
output_txt = open(output_file,'w')

#write header
for patientID in common_patientID_list:
    output_txt.write('\t%s' % patientID)
output_txt.write('\n')

for feature in common_feature_list:
    output_txt.write(feature)
    for patientID in common_patientID_list:
        value = data_dict[feature, patientID]
        output_txt.write('\t%s' % value)
    output_txt.write('\n')
output_txt.close()